In [1]:
import json
import sys
import pandas as pd
import random

In [2]:
data = pd.read_csv('/data/user_data/wenkail/llm_personality/data/big5_data_generator_test.csv')
data.head()

,message_id,message,ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
0,tweet_207056,my phones acting a little slow.. then i rememb...,1,0,0,0,1
1,blog_176919,well today was the day we got our mid-term mar...,0,1,0,1,0
2,blog_686715,being sick sucks. merry christmas eve to all.,0,0,0,0,1
3,blog_270817,"life's too short.... don't know what did it, ...",0,1,0,0,1
4,tweet_157250,poison girl is stuck in my head urgh,1,0,0,0,1


### Split Dataset

In [8]:
from sklearn.model_selection import train_test_split

train_valid, test = train_test_split(data, test_size=0.1, random_state=42)

train_size = 0.9 / (0.9 + 0.1)
train, valid = train_test_split(train_valid, train_size=train_size, random_state=42)

In [9]:
test.to_csv("alpaca_test_5_tokens_original.csv", index=False)

In [3]:
test = pd.read_csv("alpaca_test_5_tokens_original.csv")

In [11]:
num_samples = 1000
seed = 42
random.seed(seed)

sub_testset = test.sample(n=num_samples, random_state=seed)

In [12]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from nltk.tokenize import word_tokenize

def count_words(text):
    words = word_tokenize(text)
    return len(words)

[nltk_data] Downloading package punkt to /home/wenkail/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from tqdm import tqdm
tqdm.pandas()
tqdm.pandas(desc="Calculating average word length")
data['avg_word_length'] = data['message'].apply(count_words)

In [8]:
word_length_stats = data['avg_word_length'].describe()
print(word_length_stats)

count    846304.000000
mean         31.836953
std          33.870257
min           1.000000
25%          13.000000
50%          26.000000
75%          41.000000
max        4762.000000
Name: avg_word_length, dtype: float64


### Generate Alpaca

In [5]:
def generate_alpaca_dataset(df, n_words, trait):
    personality_map = {0: "low", 1: "high"}
    # traits = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]
    
    dataset = []
    
    for _, row in df.iterrows():
        message = row['message']
        words = message.split()
        
        if len(words) <= n_words:
            continue
        
        input_text = " ".join(words[:n_words])
        output_text = " ".join(words[n_words:])
        
        personality = [f"{trait.capitalize()} - {personality_map[row[f'{trait[0:3]}_z_label']]}"] 
                    #    for trait in traits]
        
        # instruction = "Help me complete the sentence with certain Big Five Personality: {}, {}, {}, {}, {}".format(*personality)
        instruction = "Help me complete the sentence with certain Big Five Personality: {}".format(*personality)
        
        item = {
            "instruction": instruction,
            "input": input_text,
            "output": output_text
        }
        
        dataset.append(item)
    
    return dataset

In [6]:
traits = ["openness", "conscientiousness", "extraversion", "agreeableness", "neuroticism"]

In [10]:
openness = generate_alpaca_dataset(data, 5, "openness")
print(openness)

In [9]:
from tqdm import tqdm
big5_train = {}
for trait in tqdm(traits):
    store[f'{trait}'] = generate_alpaca_dataset(data, 5, trait)

  0%|          | 0/5 [00:09<?, ?it/s]


KeyboardInterrupt: 

### Old Version

In [5]:
sub_testset = pd.read_csv("alpaca_test_5_tokens_subtest_1000.csv")
sub_testset = sub_testset[sub_testset['message'].apply(lambda x: len(x.split())) > 5]

In [6]:
len(sub_testset)

998

In [7]:
sub_testset.to_csv("alpaca_test_5_tokens_subtest_998.csv")

In [29]:
# alpaca_big_5_train = generate_alpaca_dataset(train, 5)
alpaca_big_5_test = generate_alpaca_dataset(sub_testset, 5)

In [ ]:
alpaca_big_5_train

In [20]:
with open('alpaca_big_five_dataset_train_5_tokens.json', 'w') as f:
    json.dump(alpaca_big_5_train, f, indent=4)

In [30]:
with open('alpaca_big_five_dataset_1000_test_5_tokens_subtestset.json', 'w') as f:
    json.dump(alpaca_big_5_test, f, indent=4)

In [4]:
with open("alpaca_big_five_dataset_1000_test_5_tokens.json", 'r') as f:
    data = json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)